In [1]:
!pip install scikit-learn
!pip install matplotlib
!pip install pandas

In [124]:
import pandas as pd

DEBUG_COUNTER = 0

df_latin = pd.read_csv('https://github.com/yarathealmighty/dumps/blob/main/latin.csv?raw=true')
df_engineer = pd.read_csv('https://github.com/yarathealmighty/dumps/blob/main/engineer.csv?raw=true')
df_latin

,Id,ParentId,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastActivityDate,Title,Tags,AnswerCount,CommentCount,ContentLicense
0,1,NaN,1,6.0,2016-02-23T18:13:46.960,59,13455.0,I was taught that one can use the '-que' suffi...,NaN,2022-05-12T21:47:46.987,"Are ""-que"" and ""et"" equivalent?",|classical-latin|meaning|vocabulary|conjunctio...,5.0,6,CC BY-SA 3.0
1,2,NaN,1,3.0,2016-02-23T18:16:00.220,44,2098.0,I learned early on that Latin has no articles....,12.0,2020-02-28T17:13:09.310,"Why is ""ille"" used in Winnie ille Pu and Hobbi...",|contemporary-latin|pronouns|articles|translat...,2.0,0,CC BY-SA 3.0
2,3,2.0,2,NaN,2016-02-23T18:16:00.220,50,NaN,"It's true that in Classical Latin, ille is a d...",12.0,2016-02-23T18:16:00.220,NaN,NaN,NaN,1,CC BY-SA 3.0
3,4,NaN,1,20.0,2016-02-23T18:18:34.973,31,1048.0,I'd like the translate the following sentence ...,12.0,2017-11-09T18:14:51.657,What gender should a predicate adjective be to...,|agreement|predicate|gender|adjective|classica...,2.0,1,CC BY-SA 3.0
4,5,NaN,1,17.0,2016-02-23T18:23:08.357,31,5066.0,Latin makes use of macrons (small lines above ...,7.0,2020-06-18T20:41:05.150,What effect should a macron have on the sound ...,|pronunciation|,4.0,1,CC BY-SA 3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16167,23432,NaN,1,NaN,2024-03-31T20:27:59.770,-1,73.0,What diphthongs are available are Unicode liga...,14839.0,2024-04-01T02:07:23.763,What diphthongs are available are Unicode liga...,|diphthong|paleography|ligatures|,1.0,6,CC BY-SA 4.0
16168,23433,NaN,1,NaN,2024-03-31T21:10:36.457,0,50.0,"In a previous question of mine, What diphthong...",14839.0,2024-03-31T22:46:26.900,How to indicate a diphthong?,|pronunciation|spelling|diphthong|ligatures|,3.0,3,CC BY-SA 4.0
16169,23434,23433.0,2,NaN,2024-03-31T21:16:18.310,2,NaN,"It isn’t traditionally used in normal writing,...",9.0,2024-03-31T21:16:18.310,NaN,NaN,NaN,0,CC BY-SA 4.0
16170,23435,23433.0,2,NaN,2024-03-31T21:21:41.503,1,NaN,"Most of them don't need to be marked, since th...",406.0,2024-03-31T21:21:41.503,NaN,NaN,NaN,0,CC BY-SA 4.0


In [117]:
def answer_conc(df):

  df_questions = df[df['ParentId'].isna()]
  df_answers = df[df['ParentId'].notna()]

  df_answers = df_answers.drop(columns=['AcceptedAnswerId','ViewCount','Tags'])

  df_questions = df_questions.merge(df_answers[['Id','Text']], how='left', left_on='AcceptedAnswerId', right_on='Id', suffixes=('', '_Answer'))
  df_questions.rename(columns={'Text_Answer': 'Answer'}, inplace=True)
  df_questions = df_questions.drop(columns='Id_Answer')
  df_answers_reduced = df_answers.loc[df_answers.groupby('ParentId')['Score'].idxmax()]
  df_no_accepted_answer = df_questions[df_questions['AcceptedAnswerId'].isna()]

  for index, row in df_no_accepted_answer.iterrows():
    question_id = row['Id']
    highest_scored_answer = df_answers_reduced[df_answers_reduced['ParentId'] == question_id]
    if not highest_scored_answer.empty:
        df_questions.at[index, 'Answer'] = highest_scored_answer.iloc[0]['Text']
  return df_questions

In [122]:
def big_clean(df):
  df = df.drop(columns=['LastActivityDate','CreationDate','PostTypeId','OwnerUserId','ContentLicense','CommentCount','AnswerCount'])
  df['Text'] = df['Title'].fillna('') + ' ' + df['Body'].fillna('').str.strip()
  df = df.drop(columns=['Body','Title'])

  questions = answer_conc(df)

  questions.rename(columns={'Text':'Question'},inplace=True)
  questions = questions.drop(columns=['ViewCount','AcceptedAnswerId','ParentId','Score'])
  return questions

In [123]:
latin = big_clean(df_latin)
# ~1000 questions dropped
# latin = latin.dropna(subset=['Answer'])
latin

,Id,Tags,Question,Answer
0,1,|classical-latin|meaning|vocabulary|conjunctio...,"Are ""-que"" and ""et"" equivalent? I was taught t...","The way I was taught was that, as a general r..."
1,2,|contemporary-latin|pronouns|articles|translat...,"Why is ""ille"" used in Winnie ille Pu and Hobbi...","It's true that in Classical Latin, ille is a ..."
2,4,|agreement|predicate|gender|adjective|classica...,What gender should a predicate adjective be to...,Bennett's New Latin Grammar ( this link will ...
3,5,|pronunciation|,What effect should a macron have on the sound ...,"In most modern texts, the whole purpose of us..."
4,7,|etymology|particle|hebrew|,"Does ""ad"" have its origin in Hebrew/Semitic la...","No, the similarity is almost certainly accide..."
...,...,...,...,...
6834,23420,|english-to-latin-translation|,Four more loaves please This new question: How...,"In Nicholas Oulton's Book II, p.50: ""plures c..."
6835,23425,|latin-to-english-translation|mathematics|,"Mathematical Latin Help So, I'm a PhD student ...","I understand ""ut quotiens sit integer complex..."
6836,23429,|latin-to-english-translation|translation-check|,Does “interranima” mean “inner soul”? I came a...,"No. Anima is the Latin word for soul, apart f..."
6837,23432,|diphthong|paleography|ligatures|,What diphthongs are available are Unicode liga...,NaN


In [126]:
engineer = big_clean(df_engineer)
# ~2000 questions dropped
# engineer = engineer.dropna(subset=['Answer'])
engineer

,Id,Tags,Question,Answer
0,1,|ipsec|ike|vpn|,What's the difference between IKE and ISAKMP? ...,"ISAKMP is part of IKE. (IKE has ISAKMP, SKEME..."
1,3,|qos|,Quality of Service based on Layer 7 traffic Ho...,I think this depends heavily on the platform ...
2,4,|cisco-catalyst|bgp|,"Troubleshooting ""Down BGP"" connections Our net...",172259: May 6 14:43:06: %BGP-3-NOTIFICATION:...
3,5,|ospf|security|,Securing OSPF with only passive interfaces I k...,One issue is that authentication ensures that...
4,6,|juniper|policing|juniper-ex|,Aggregate Family Rate Limits on Juniper EX Try...,"You are looking for the ""Logical Interface (A..."
...,...,...,...,...
17511,85387,|bgp|,Is there a reason to keep the BGP communities ...,Communities can be used for all sorts of thin...
17513,85389,|bgp|juniper|,Does Junos prefer route with higher localpref ...,This totally depends on what you mean by 'bec...
17515,85393,|cisco-asa|vpn|ipsec|split-tunneling|,Cisco ASA L2TP/IPsec split-tunnel not working ...,Perhaps the client does not properly support ...
17518,85398,|switch|network|networking|layer3|,Core Switches Redundancy Problem I have networ...,"There are several ways to do this, depending ..."


In [ ]:
# stuff to try OHE/BOW
# 2d ábrázolás
# BERT változatok
# akár neur.háló
# cluster és tag korreláció